In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///tests.db')

In [3]:
import pandas as pd
from loguru import logger

In [4]:
def create_test_set(*, test_path: str, testcase_path: str) -> pd.DataFrame:
    test_df = pd.read_json(test_path, lines=True)
    # Use vectorized string concatenation
    test_df["sample_id"] = test_df["id"].astype(str) + "_" + test_df["sol"].astype(str)

    logger.debug(f"Test Dataset has {len(test_df)} entries")

    testcase_df = pd.read_json(testcase_path, lines=True).set_index("avatar_id")
    test_filtered = test_df[test_df.id.isin(testcase_df.index)]

    logger.debug(f"Testcase Dataset has {len(testcase_df)} entries")

    # Perform a left merge on the 'id' column of test_df and the index of testcase_df
    merged_df = pd.merge(
        test_filtered, testcase_df, left_on="id", right_index=True, how="left"
    )

    logger.info(f"Merged dataset has {len(merged_df)} entries")
    logger.debug(merged_df.head(2))
    return merged_df.set_index("sample_id")

In [5]:
df = create_test_set(test_path="test.java-python.id_code", testcase_path="combined_id2tests.jsonl")

2024-01-04 14:42:26.210 | DEBUG    | __main__:create_test_set:6 - Test Dataset has 3046 entries
2024-01-04 14:42:26.262 | DEBUG    | __main__:create_test_set:11 - Testcase Dataset has 1314 entries
2024-01-04 14:42:26.264 | INFO     | __main__:create_test_set:18 - Merged dataset has 431 entries
2024-01-04 14:42:26.265 | DEBUG    | __main__:create_test_set:19 -                   id  sol                                        python_code  \
9   atcoder_ABC175_D    0  N , K = map ( int , input ( ).split ( ) )\nP =...   
12  codeforces_581_B    0  n = int ( input ( ) )\narr = list ( map ( int ...   

                                python_code_tokenized           sample_id  \
9   N , K = map ( int , input ( ) . split ( ) ) NE...  atcoder_ABC175_D_0   
12  n = int ( input ( ) ) NEW_LINE arr = list ( ma...  codeforces_581_B_0   

                                               inputs  \
9   [ABC175/D/in/29.txt, ABC175/D/in/15.txt, ABC17...   
12  [581_B/samples/10_input.txt, 581_B/samples/31_.

In [6]:
import os
import json


def generate(row):
    if row.id.startswith('atcoder'):
        path = '../../test_cases/atcoder_test_cases'
    else:
        path = '../../test_cases/description2code_current/codeforces'

    # Initialize the lists outside the loop
    data = []

    for i, o in zip(row['inputs'], row['outputs']):
        input_file = os.path.join(path, i)
        output_file = os.path.join(path, o)
        obj = {
            'test_case_name': i
        }
        with open(input_file, 'r') as i_f:
            obj['input'] = i_f.read()

        with open(output_file, 'r') as o_f:
            obj['output'] = o_f.read()

        data.append(obj)
    row['tcs'] = json.dumps(data)
    return row


# Assuming df is your DataFrame
df = df.apply(generate, axis=1)

In [7]:
df = df[['id', 'python_code', 'tcs']]

In [8]:
df

,id,python_code,tcs
sample_id,,,
atcoder_ABC175_D_0,atcoder_ABC175_D,"N , K = map ( int , input ( ).split ( ) )\nP =...","[{""test_case_name"": ""ABC175/D/in/29.txt"", ""inp..."
codeforces_581_B_0,codeforces_581_B,n = int ( input ( ) )\narr = list ( map ( int ...,"[{""test_case_name"": ""581_B/samples/10_input.tx..."
codeforces_253_A_0,codeforces_253_A,"with open ( ""input.txt"" , ""r"" ) as inp :\n ...","[{""test_case_name"": ""253_A/samples/10_input.tx..."
atcoder_AGC003_C_0,atcoder_AGC003_C,N = int ( input ( ) )\nA = [ int ( input ( ) )...,"[{""test_case_name"": ""AGC003/C/in/15.txt"", ""inp..."
atcoder_ABC127_B_0,atcoder_ABC127_B,"r , D , x = map ( int , input ( ).split ( ) )\...","[{""test_case_name"": ""ABC127/B/in/15.txt"", ""inp..."
...,...,...,...
codeforces_266_A_0,codeforces_266_A,n = int ( input ( ) )\ns = input ( )\ncount = ...,"[{""test_case_name"": ""266_A/samples/10_input.tx..."
atcoder_ARC075_C_0,atcoder_ARC075_C,"class BIT :\n def __init__ ( self , n ) :\n...","[{""test_case_name"": ""ARC075/C/in/b10"", ""input""..."
codeforces_597_A_0,codeforces_597_A,"k , a , b = map ( int , input ( ).split ( ) )\...","[{""test_case_name"": ""597_A/samples/10_input.tx..."


df.to_sql?

In [9]:
df.to_sql(name='tcs', con=engine)

431

In [29]:
engine

Engine(sqlite:///testcases.db)

In [10]:
s=ks

NameError: name 'ks' is not defined

In [11]:
ks

NameError: name 'ks' is not defined

In [12]:
ls

__init__.py                 sampled_outputs/
__pycache__/                sampler/
baseline_analysis.ipynb     sampler.py
calculate_baseline.py       start_slurm.py
codellama-13b-inst.gguf     test.java-python.id_code
combined_id2tests.jsonl     testcases.db
evaluator.py                tests.db
finetuning/                 tmp/
generate_samples.py         utils.py
generate_testcase_db.ipynb  workers/
output/
